In [1]:
import re
import jinja2
from anthropic import AnthropicBedrock
from core import stages

In [2]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:
environment = jinja2.Environment()

In [4]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [5]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [6]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [7]:
typespec

{'reasoning': 'For a gym tracking bot, users will likely send messages like:\n"I did 3 sets of bench press with 135lbs today" or \n"What\'s a good 30-minute workout for chest using dumbbells?"\nKey elements to track:\n- Exercises with sets, reps, and weights\n- Available equipment for workout planning\n- Time duration for workouts\n- Progress tracking over time\n- Workout categories (muscle groups)\n\nMain functions needed:\n- Record completed exercises\n- Get workout suggestions based on constraints\n- Track progress for specific exercises\n- View exercise history\n\nThe LLM can parse natural language into structured data:\n"I did 3 sets of bench press with 135lbs" -> \nrecordExercise({name: "bench press", sets: 3, weight: 135, ...})',
 'typespec_definitions': 'model Exercise {\n  name: string;\n  muscleGroup: string;\n  sets: int32;\n  reps: int32;\n  weight?: float32;\n  equipment: string;\n  date: Date;\n}\n\nmodel WorkoutConstraints {\n  duration: int32; // in minutes\n  equipment

In [8]:
print(typespec["reasoning"])

For a gym tracking bot, users will likely send messages like:
"I did 3 sets of bench press with 135lbs today" or 
"What's a good 30-minute workout for chest using dumbbells?"
Key elements to track:
- Exercises with sets, reps, and weights
- Available equipment for workout planning
- Time duration for workouts
- Progress tracking over time
- Workout categories (muscle groups)

Main functions needed:
- Record completed exercises
- Get workout suggestions based on constraints
- Track progress for specific exercises
- View exercise history

The LLM can parse natural language into structured data:
"I did 3 sets of bench press with 135lbs" -> 
recordExercise({name: "bench press", sets: 3, weight: 135, ...})


In [9]:
print(typespec["typespec_definitions"])

model Exercise {
  name: string;
  muscleGroup: string;
  sets: int32;
  reps: int32;
  weight?: float32;
  equipment: string;
  date: Date;
}

model WorkoutConstraints {
  duration: int32; // in minutes
  equipment: string[];
  targetMuscles: string[];
  difficulty: "beginner" | "intermediate" | "advanced";
}

model WorkoutPlan {
  exercises: Exercise[];
  totalDuration: int32;
  difficulty: string;
  caloriesBurn: int32;
}

interface GymTracker {
  @llm_func(2)
  recordExercise(exercise: Exercise): void;

  @llm_func(3)
  suggestWorkout(constraints: WorkoutConstraints): WorkoutPlan;

  @llm_func(1)
  getProgress(exerciseName: string, startDate: Date, endDate: Date): Exercise[];

  @llm_func(1)
  listWorkoutHistory(date: Date): Exercise[];
}


In [10]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [11]:
print(drizzle["drizzle_schema"])

import { integer, serial, text, timestamp, real, pgTable, pgEnum } from "drizzle-orm/pg-core";

// Enums
export const difficultyEnum = pgEnum("difficulty", ["beginner", "intermediate", "advanced"]);

// Equipment table
export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").unique().notNull()
});

// Muscle groups table
export const muscleGroupsTable = pgTable("muscle_groups", {
  id: serial("id").primaryKey(),
  name: text("name").unique().notNull()
});

// Exercises table
export const exercisesTable = pgTable("exercises", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  muscleGroupId: integer("muscle_group_id").references(() => muscleGroupsTable.id),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight"),
  equipmentId: integer("equipment_id").references(() => equipmentTable.id),
  date: timestamp("date").notNull(),
  createdAt: timestamp("created_at").defaultNow()
});

// W

In [12]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [13]:
print(pre_response.content[0].text)

<instructions>
The listWorkoutHistory function requires a single Date parameter and returns an array of Exercise records for that specific date. The date should be interpreted from natural language inputs and defaulted to current date if not specified. Time component of the date can be ignored. The function returns all exercises recorded for the given date.
</instructions>

<examples>
    <example>
        <input>Show me my workouts from yesterday</input>
        <output>{
    "date": "2024-01-16"  // Assuming today is 2024-01-17
}</output>
    </example>

    <example>
        <input>What exercises did I do on January 10th?</input>
        <output>{
    "date": "2024-01-10"
}</output>
    </example>

    <example>
        <input>Show today's workout history</input>
        <output>{
    "date": "2024-01-17"  // Assuming today is 2024-01-17
}</output>
    </example>

    <example>
        <input>Get my exercises from last Monday</input>
        <output>{
    "date": "2024-01-15"  // As

In [14]:
pre_processors

{'recordExercise': {'instructions': 'For the recordExercise function:\n1. Exercise name should be extracted from common exercise terminology\n2. If weight is not specified, it should be omitted from the output\n3. If date is not specified, use current date\n4. Equipment should be inferred from exercise type if not specified\n5. If sets/reps are not specified, default to 1 set\n6. Muscle group should be inferred from exercise name if not specified',
  'examples': [('bench press 3x10 with 225 lbs',
    '{\n    "name": "bench press",\n    "muscleGroup": "chest",\n    "sets": 3,\n    "reps": 10,\n    "weight": 225,\n    "equipment": "barbell",\n    "date": "2024-01-20T12:00:00Z"\n}'),
   ('did 15 pushups this morning',
    '{\n    "name": "pushup",\n    "muscleGroup": "chest",\n    "sets": 1,\n    "reps": 15,\n    "equipment": "bodyweight",\n    "date": "2024-01-20T08:00:00Z"\n}'),
   ('lat pulldown 4 sets of 12 reps at 60kg',
    '{\n    "name": "lat pulldown",\n    "muscleGroup": "back",

In [15]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)

In [16]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { \n    exercisesTable, \n    equipmentTable, \n    muscleGroupsTable \n} from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: {\n    name: string;\n    muscleGroup: string;\n    sets: number;\n    reps: number;\n    weight?: number;\n    equipment: string;\n    date: Date;\n}): Promise<void> => {\n    // Get or create equipment\n    let equipmentRecord = await db.query.equipmentTable.findFirst({\n        where: eq(equipmentTable.name, exercise.equipment)\n    });\n\n    if (!equipmentRecord) {\n        const [newEquipment] = await db.insert(equipmentTable)\n            .values({ name: exercise.equipment })\n            .returning();\n        equipmentRecord = newEquipment;\n    }\n\n    // Get or create muscle group\n    let muscleGroupRecord = await db.query.muscleGroupsTable.findFirst({\n        where: eq(muscleGroupsTable.name, exercise.muscleGroup)\n    });\n\n

In [20]:
with open("core/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [21]:
hdl_key = "recordExercise"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [22]:
print(file_content)

import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { 
    exercisesTable, 
    equipmentTable, 
    muscleGroupsTable 
} from '../db/schema/application';
import { eq } from 'drizzle-orm';

const handle = async (exercise: {
    name: string;
    muscleGroup: string;
    sets: number;
    reps: number;
    weight?: number;
    equipment: string;
    date: Date;
}): Promise<void> => {
    // Get or create equipment
    let equipmentRecord = await db.query.equipmentTable.findFirst({
        where: eq(equipmentTable.name, exercise.equipment)
    });

    if (!equipmentRecord) {
        const [newEquipment] = await db.insert(equipmentTable)
            .values({ name: exercise.equipment })
            .returning();
        equipmentRecord = newEquipment;
    }

    // Get or create muscle group
    let muscleGroupRecord = await db.query.muscleGroupsTable.findFirst({
        where: eq(muscleGroupsTable.name, 

In [14]:
print(handlers["suggestWorkout"]["handler"])

import { db } from "../db";
import { 
  equipmentTable, 
  workoutRoutinesTable, 
  exercisesTable,
  routineExercisesTable,
  routineEquipmentTable,
  targetMusclesTable,
  routineTargetMusclesTable
} from '../db/schema/application';
import { eq, inArray } from 'drizzle-orm';

const handle = async (constraints: WorkoutConstraints): Promise<WorkoutRoutine> => {
  // Get equipment IDs based on provided constraints
  const equipmentIds = await db
    .select({ id: equipmentTable.id })
    .from(equipmentTable)
    .where(inArray(equipmentTable.name, constraints.equipment.map(e => e.name)))
    .execute();

  // Get muscle IDs based on provided target muscles
  const muscleIds = await db
    .select({ id: targetMusclesTable.id })
    .from(targetMusclesTable)
    .where(inArray(targetMusclesTable.name, constraints.targetMuscles))
    .execute();

  // Find suitable workout routines that match the constraints
  const routines = await db
    .select()
    .from(workoutRoutinesTable)
    .wh

In [ ]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [6]:
ts_defs

{'typespec_definitions': 'model Dish {\n    name: String\n    ingredients: Ingredient[]\n}\n\nmodel Ingredient {\n    name: String\n    calories: Int\n}\n\ninterface DietBot {\n    @llm_func("pre", 1)\n    recordDish(dish: Dish): void;\n    @llm_func("wrap", 1)\n    listDishes(from: Date, to: Date): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [4]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [5]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [1]:
from shutil import copytree, ignore_patterns

In [4]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [5]:
setup_project('test')

In [6]:
from core.stages import router

In [7]:
funcs = router.parse_output(router.PROMPT)

In [8]:
funcs

{'functions': [{'name': 'recordDish',
   'description': '\nLog user\'s dish. Examples:\n- "I ate a burger."\n- "I had a salad for lunch."\n- "Chili con carne"\n'},
  {'name': 'listDishes',
   'description': '\nList user\'s dishes. Examples:\n- "What did I eat yesterday?"\n- "Show me my meals for last week."\n'}]}

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
